In [2]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [22]:
import numpy as np
import pandas as pd
from pathlib import Path
import swifter

import sys
sys.path.append('../src')

import matplotlib.pyplot as plt
from tqdm import tqdm

import utils
from dataset import M5Dataset
import preprocessing
import features
from evaluater import WRMSSEEvaluator

In [27]:
import dask.dataframe as dd

In [5]:
melted_df = utils.load_pickle(utils.FEATURE_DIR / 'melted.pkl')

In [61]:
ddf = dd.from_pandas(melted_df, npartitions=100)

In [62]:
ddf.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,...,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,item_id_1,item_id_2,item_id_3,all_id
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,...,unknown,unknown,0,0,0,NaN,HOBBIES_1,0,01,all
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,...,unknown,unknown,0,0,0,NaN,HOBBIES_1,0,02,all
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,...,unknown,unknown,0,0,0,NaN,HOBBIES_1,0,03,all
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,...,unknown,unknown,0,0,0,NaN,HOBBIES_1,0,04,all
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,...,unknown,unknown,0,0,0,NaN,HOBBIES_1,0,05,all


In [13]:
ddf.groupby(['id']).agg({'demand': 'mean'})

#,id,demand
0,FOODS_3_203_CA_4_validation,1.6295723853683668
1,FOODS_3_204_CA_4_validation,0.3930963420917053
2,FOODS_3_205_CA_4_validation,0.14013395157135497
3,FOODS_3_206_CA_4_validation,0.386913961875322
4,FOODS_3_207_CA_4_validation,0.779495105615662
...,...,...
"30,485",FOODS_3_198_CA_4_validation,0.22102009273570325
"30,486",FOODS_3_199_CA_4_validation,0.4456465739309634
"30,487",FOODS_3_200_CA_4_validation,0.2807831014940752
"30,488",FOODS_3_201_CA_4_validation,1.2601751674394641


In [15]:
ddf['demand'].shift(1)

AttributeError: 'Expression' object has no attribute 'shift'

In [16]:
group_ids = ('all_id', 'state_id', 'store_id', 'cat_id', 'dept_id', 'item_id',
                 ['state_id', 'cat_id'],  ['state_id',
                                           'dept_id'], ['store_id', 'cat_id'],
                 ['store_id', 'dept_id'], ['item_id', 'state_id'], ['item_id', 'store_id'])

In [20]:
melted_df['all_id'] = 'all'

In [23]:
%%time
for group_id in tqdm(group_ids):
    if isinstance(group_id, str):
        group_id = [group_id]
    _ = melted_df.groupby(group_id + ['date'])['demand'].mean().groupby(group_id)

100%|██████████| 12/12 [02:43<00:00, 13.59s/it]

CPU times: user 2min 20s, sys: 52.1 s, total: 3min 12s
Wall time: 2min 43s


In [24]:
%%time
for group_id in tqdm(group_ids):
    if isinstance(group_id, str):
        group_id = [group_id]
    _ = melted_df.groupby(group_id + ['date'])['demand'].mean().groupby(group_id).apply(lambda x: x.shift(1))

100%|██████████| 12/12 [03:38<00:00, 18.21s/it]

CPU times: user 6min 20s, sys: 1min 16s, total: 7min 36s
Wall time: 3min 38s


In [60]:
%%time
for _ in tqdm(range(10)):
    melted_df.groupby(['id', 'date'])['demand'].mean().groupby(['id']).apply(lambda x: x.shift(2))

CPU times: user 13min 22s, sys: 7min 54s, total: 21min 16s
Wall time: 11min 26s


## dask

In [63]:
%%time
for _ in tqdm(range(10)):
    ddf.groupby(['id', 'date']).agg({'demand':'mean'}).compute().reset_index().groupby(['id']).apply(lambda x: x.shift(2))

100%|██████████| 10/10 [23:46<00:00, 142.69s/it]

CPU times: user 2h 14min 48s, sys: 3h 35min 48s, total: 5h 50min 37s
Wall time: 23min 46s


In [32]:
ddf.groupby(['id', 'date']).agg({'demand':'mean'}).compute().reset_index().groupby

SyntaxError: unexpected EOF while parsing (<unknown>, line 4)